# Logistic回归
其分类的结果输出是有限的离散值，基本思想是在空间中构造一个合理的超平面，把空间区域划分为两个子空间，每一种类别数据都在平面的某一侧。
Logistic回归通过使用逻辑函数（也称为sigmoid函数）来估计概率，从而预测分类结果。其核心思想是根据输入特征计算出一个概率值，然后根据这个概率值进行分类决策。
    Logistic回归使用sigmoid函数将线性回归的输出映射到(0,1)区间：
    σ(z) = 1 / (1 + e^(-z))
        其中z是输入特征的线性组合：
        z = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ
# Logistic回归模型可以表示为：
P(Y=1|X) = 1 / (1 + e^(-(β₀ + β₁x₁ + ... + βₙxₙ)))
其中：
    P(Y=1|X)是在给定特征X下Y=1的概率
    β₀是截距项
    β₁到βₙ是各特征的系数
# 参数估计
Logistic回归通常使用极大似然估计(MLE)来估计参数。目标是找到一组参数，使得观测数据的似然性最大化。
在Logistic回归中，MLE用于估计回归系数β。对数似然函数为：
ℓ(β) = ∑ [yᵢ log p(xᵢ) + (1-yᵢ) log(1-p(xᵢ))]
其中p(xᵢ) = 1/(1+exp(-βᵀxᵢ))。通过数值优化方法(如牛顿法、梯度下降等)最大化这个对数似然函数来估计β。

# 分组数据情形下参数的最小二乘估计
在对因变量进行的n次观测yi，如果在相同的x(i)处进行了多次重复观测，这种结构的数据称为分组数据。可用样本比例对概率p进行估计，对应的x(i)的概率估计值记作(估计值)pi,
yi* = ln(pi/1-pi)----->yi* = β₀+Σβᵢxᵢj

# 可以通过statsmodels库函数求解

In [4]:
#例一
import numpy as np
import statsmodels.api as sm

a = np.loadtxt('Pdata12_7_1.txt')
x = a[:,0]
pi = a[:,2]/a[:,1]
X = sm.add_constant(x)
yi = np.log(pi/(1-pi))  #yi* = ln(pi/1-pi)
md = sm.OLS(yi,X).fit()  #构建并拟合模型
print(md.summary())
b = md.params  #提出所有的回归系数
# print(b)
p0 = 1/(1+np.exp(-np.dot(b, [1,9])))  #构造Logistic回归方程
"""
np.dot(b, [1, 9])
b 是 Logistic 回归的系数向量，假设 b = [β₀, β₁]（截距 + 斜率）。
[1, 9] 是特征向量，其中 1 是截距项（x₀），9 是某个特征值（x₁）。
np.dot(b, [1, 9]) 计算的是 线性组合：z=β0·1+β1⋅9
"""
print('所求的概率p0=%.4f' %p0)
np.savetxt('Pdata12_7_2.txt',b)  #把回归系数保存到文本文件

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     85.42
Date:                Sun, 27 Jul 2025   Prob (F-statistic):           3.59e-05
Time:                        16:13:05   Log-Likelihood:                 6.6829
No. Observations:                   9   AIC:                            -9.366
Df Residuals:                       7   BIC:                            -8.971
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8863      0.102     -8.653      0.0

# Logistic模型的应用
1. 在流行病学中，经常需要研究某一疾病发生与不发生的可能性大小，如一个人得流行性感冒相对于不得流行性感冒的可能性是多少，对此常用赔率来度量
    (1) 赔率的定义
    一个随机事件A发生的概率与其不发生的概率之比值称为事件A的赔率，记为odds(A),即odds(A) = P(A)/(1-P(A)).
        用赔率可以很好的度量一些经济现象发生与否的可能性大小
    (2) 赔率比的定义
    随机事件A的赔率与随机事件B的赔率之比值称为事件A对事件B的赔率比，记为OR(A,B) = odds(A) / odds(B)

In [6]:
#例一续
import numpy as np

b = np.loadtxt('Pdata12_7_2.txt')
odds_9 = np.exp(np.dot(b,[1,9]))  #$记算x=9时的赔率
odds_9vs8 = np.exp(np.dot([1,9],b))/np.exp(np.dot(b,[1,8]))
print('odds_9=%.4f\nodds_9vs8=%.4f' %(odds_9, odds_9vs8))

odds_9=1.6752
odds_9vs8=1.1686


# 通过statsmodels库函数求解并进行预测

1. 牛顿法原理
牛顿法通过使用二阶导数信息（Hessian矩阵）来加速收敛。对于函数f(θ)，其迭代公式为：
    θ^(t+1) = θ^(t) - [H(θ^(t))]⁻¹ ∇f(θ^(t))
其中：
    ∇f(θ)是梯度向量（一阶导数）
    H(θ)是Hessian矩阵（二阶导数）

在Logistic回归中的应用
对于Logistic回归的对数似然函数ℓ(β)，牛顿法的迭代步骤为：
β^(t+1) = β^(t) + (X'WX)⁻¹ X'(y - p)
其中：
    X是设计矩阵
    W是对角权重矩阵，Wᵢᵢ = pᵢ(1-pᵢ)
    p是预测概率向量
    y是响应变量向量

2. bfgs法
BFGS（Broyden-Fletcher-Goldfarb-Shanno）是最流行的拟牛顿优化算法之一，特别适用于Logistic回归等光滑非线性优化问题。
    1. BFGS算法原理
BFGS是一种拟牛顿法，通过迭代近似Hessian矩阵（而不是直接计算），具有以下特点：
    不需要计算二阶导数，仅使用一阶导数（梯度）信息
    保持Hessian近似矩阵的正定性，确保下降方向
    具有超线性收敛速度
    2. 迭代公式
BFGS的更新步骤如下：
    计算搜索方向：dₖ = -Bₖ⁻¹∇f(θₖ)
    线搜索确定步长αₖ
    更新参数：θₖ₊₁ = θₖ + αₖdₖ
    更新Hessian近似矩阵Bₖ₊₁
其中Hessian近似矩阵B的BFGS更新公式为：
    Bₖ₊₁ = Bₖ + (yₖyₖᵀ)/(yₖᵀsₖ) - (BₖsₖsₖᵀBₖ)/(sₖᵀBₖsₖ)
    其中sₖ = θₖ₊₁ - θₖ，yₖ = ∇f(θₖ₊₁) - ∇f(θₖ)
在Logistic回归中的应用
    优势
        内存效率：对于高维问题，可以使用有限内存版本(L-BFGS)
        数值稳定性：不需要直接计算Hessian矩阵
        适应性强：适用于各种规模的数据集
    实现步骤
        定义对数似然函数ℓ(β)及其梯度
        初始化参数β₀和Hessian近似矩阵B₀（通常取单位矩阵）
        迭代更新直至收敛

In [11]:
import numpy as np
import statsmodels.api as sm

a = np.loadtxt('Pdata12_9.txt')
n = a.shape[1]  #提取矩阵的列数
x = a[:,:n-1]
y = a[:,n-1]
md = sm.Logit(y,x)
md = md.fit(method='bfgs')  #这里必须使用bfgs法，使用默认牛顿法会出错
"""
1. 牛顿法需要计算Hessian矩阵(二阶导数矩阵)的逆。如果：
    数据存在完全分离(perfect separation)(某些特征能完美预测分类)
    存在高度相关的特征(导致矩阵奇异)
    样本量过少或特征过多
Hessian矩阵可能无法求逆,导致报错
2. 初始值敏感
    牛顿法对初始参数值敏感，如果初始值离最优解太远，可能无法收敛。

BFGS是一种拟牛顿法
    它通过近似Hessian矩阵(而不是直接计算)，避免了矩阵求逆的数值不稳定性。
    即使Hessian矩阵不可逆,BFGS仍可能收敛。
更鲁棒的优化
    BFGS使用一阶导数(梯度)信息,对初始值的依赖性较低。
    适合处理非凸优化问题（如逻辑回归的对数似然函数）。
"""
print(md.params, '\n---------------\n')
print(md.summary())
#求预测值
pre = md.predict([[-49.2,-17.2,0.3], [40.6,26.4,1.8]])
print(pre)

Optimization terminated successfully.
         Current function value: 0.000002
         Iterations: 18
         Function evaluations: 21
         Gradient evaluations: 21
[-0.34967435  3.22899624  2.23724381] 
---------------

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   20
Model:                          Logit   Df Residuals:                       17
Method:                           MLE   Df Model:                            2
Date:                Sun, 27 Jul 2025   Pseudo R-squ.:                   1.000
Time:                        18:44:22   Log-Likelihood:            -4.6356e-05
converged:                       True   LL-Null:                       -13.863
Covariance Type:            nonrobust   LLR p-value:                 9.537e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------

# 使用sklearn 库函数进行求解并进行预测

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

a = np.loadtxt('Pdata12_9.txt')
n = a.shape[1]
x = a[:,:n-1]
y = a[:,n-1]
md = LogisticRegression(solver='lbfgs').fit(x,y)
print(md.intercept_, md.coef_)  #输出截距项和斜率
print(md.predict(x))  #检验预测模型
print(md.predict([[-49.2,-17.2,0.3], [40.6,26.4,1.8]]))  #求预测值

[-0.39060121] [[-0.05073412  0.67071015  0.10511888]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0. 1.]
